In [3]:
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
import numpy as np

import wandb
from wandb.keras import WandbCallback

from PIL import Image

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [4]:
## Automate the building of CNN
def createCNN(num_filters=32, filter_multiplier=1, dropout=0.2, batch_norm=False, filter_size=(7,7), dense_size=128, num_classes=10):
    model = Sequential()
    
    if batch_norm:
        for i in range(5):
            if i <= 2:
                model.add(Conv2D(num_filters, filter_size))
            else:
                model.add(Conv2D(num_filters, (3,3)))
            model.add(BatchNormalization())
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            num_filters = int(num_filters * filter_multiplier)

    else:
        for i in range(5):
            if i <= 2:
                model.add(Conv2D(num_filters, filter_size))
            else:
                model.add(Conv2D(num_filters, (3,3)))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            num_filters = int(num_filters * filter_multiplier)
    
    model.add(Flatten())
    model.add(Dense(dense_size))
    model.add(Dropout(dropout))
    model.add(Activation("relu"))
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))

    return model;

In [5]:
## Prepare the dataset for training
def prepare_dataset(DATA_DIR="D:\Documents\Engineering - IITM\8th Semester\CS6910 - Deep Learning\inaturalist_12K", augment_data=False):
    train_dir = os.path.join(DATA_DIR, "train")
    test_dir = os.path.join(DATA_DIR, "val")

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(200, 200), batch_size=256, subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(200, 200), batch_size=256, subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(200, 200), batch_size=256)
    
    return train_generator, val_generator, test_generator;

In [6]:
def setRunName(num_filters=32, filter_multiplier=1, augment_data=False, dropout=0.2, batch_norm=False):
    
    augment_data_options = {True: "Y", False: "N"}
    batch_norm_options = {True: "Y", False: "N"}

    run_name = "_".join(["num", str(num_filters), "org", str(filter_multiplier), "aug", augment_data_options[augment_data],
                      "drop", str(dropout), "norm", batch_norm_options[batch_norm]])
    
    return run_name;

In [7]:
def train():

    config_defaults = {
        "num_filters": 32,
        "filter_multiplier": 1,
        "augment_data": False,
        "dropout": 0.2,
        "batch_norm": False,
        "epochs": 5
    }

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config
    wandb.run.name = setRunName(config.num_filters, config.filter_multiplier, config.augment_data, config.dropout, config.batch_norm)

    train_generator, val_generator, test_generator = prepare_dataset(augment_data=config.augment_data)
    model = createCNN(num_filters=config.num_filters, filter_multiplier=config.filter_multiplier,
                      dropout=config.dropout, batch_norm=config.batch_norm)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics="categorical_accuracy")
    model.fit(train_generator, epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])

In [8]:
sweep_config = {
    "name": "Testing filter size",
    "description": "Testing good filter size variation in the CNN",
    "metric": "Val Accuracy",
    "method": "grid",
    "project": "CS6910_Assignment2",
    "parameters": {
        "num_filters": {
            "values": [32]
        },
        "filter_multiplier": {
            "values": [1]
        },
        "augment_data": {
            "values": [True]
        },
        "dropout": {
            "values": [0.2]
        },
        "batch_norm": {
            "values": [False]
        }
    }
}

# creating the sweep
sweep_id = wandb.sweep(sweep_config, project="CS6910_Assignment2")

Create sweep with ID: yay7ot6o
Sweep URL: https://wandb.ai/avyay/cs6910_assignment2/sweeps/yay7ot6o


In [9]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: ewmtzzud with config:
wandb: 	augment_data: True
wandb: 	batch_norm: False
wandb: 	dropout: 0.2
wandb: 	filter_multiplier: 1
wandb: 	num_filters: 32
wandb: Currently logged in as: avyay (use `wandb login --relogin` to force relogin)
C:\Users\rao_a\Miniconda3\envs\PythonCPU\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.10.24 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: wandb version 0.10.24 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
 1/36 [..............................] - ETA: 0s - loss: 2.2917 - categorical_accuracy: 0.1211

W0402 17:24:55.292282 12512 deprecation.py:323] From C:\Users\rao_a\Miniconda3\envs\PythonCPU\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


36/36 [==============================] - 802s 22s/step - loss: 2.2995 - categorical_accuracy: 0.1116 - val_loss: 2.2797 - val_categorical_accuracy: 0.1341
Epoch 2/5
36/36 [==============================] - 724s 20s/step - loss: 2.2595 - categorical_accuracy: 0.1381 - val_loss: 2.2183 - val_categorical_accuracy: 0.1632
Epoch 3/5
36/36 [==============================] - 705s 20s/step - loss: 2.2277 - categorical_accuracy: 0.1598 - val_loss: 2.2200 - val_categorical_accuracy: 0.1842
Epoch 4/5
36/36 [==============================] - 700s 19s/step - loss: 2.2103 - categorical_accuracy: 0.1761 - val_loss: 2.1909 - val_categorical_accuracy: 0.1942
Epoch 5/5
36/36 [==============================] - 712s 20s/step - loss: 2.1921 - categorical_accuracy: 0.1863 - val_loss: 2.1712 - val_categorical_accuracy: 0.1832


epoch,4
loss,2.19209
categorical_accuracy,0.18633
val_loss,2.17118
val_categorical_accuracy,0.18318
_runtime,3825
_timestamp,1617368226
_step,4
best_val_loss,2.17118
best_epoch,4


epoch,▁▃▅▆█
loss,█▅▃▂▁
categorical_accuracy,▁▃▆▇█
val_loss,█▄▄▂▁
val_categorical_accuracy,▁▄▇█▇
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
